In [ ]:
using POMDPs, QuickPOMDPs, POMDPModelTools, POMDPSimulators, QMDP

In [ ]:
# tracts
# demographics of every tract

In [ ]:
# probability of self-reporting increase per tract, based on demographics of tract

In [ ]:
# probability of a visit increasing percentage in that tract -> fixed mean + stddev based on demographics

In [ ]:
# cost of visiting = -10  (-c)

In [ ]:
# reward at finish = a * avg(percent_completion) + b * stdev(percent_completion_of_each_demographic_group)

In [ ]:
m = QuickPOMDP(
    # n tracts, m percentage
    # states = t, all combinations of completion percentages discretized -> t * n^m
    # actions = census tract to visit,  # ["left", "right", "listen"],
    # reward(nonterminalstate, action, sp) = if action == skip: return 0 else: return -10,
    # reward(terminalstate, action, sp) = percent completion, representativeness
    
    # observations = ["left", "right"],
    initialstate = Uniform(["left", "right"]),
    discount = 1.0,

    transition = function (s, a)
        if a == "listen"
            return Deterministic(s) # tiger stays behind the same door
        else # a door is opened
            return Uniform(["left", "right"]) # reset
        end
    end,

    # observation = function (s, a, sp)
    #     if a == "listen"
    #         if sp == "left"
    #             return SparseCat(["left", "right"], [0.85, 0.15]) # sparse categorical distribution
    #         else
    #             return SparseCat(["right", "left"], [0.85, 0.15])
    #         end
    #     else
    #         return Uniform(["left", "right"])
    #     end
    # end,

    reward = function (s, a)
        if a == "listen"
            return -1.0
        elseif s == a # the tiger was found
            return -100.0
        else # the tiger was escaped
            return 10.0
        end
    end
)

solver = QMDPSolver()
policy = solve(solver, m)

rsum = 0.0
for (s,b,a,o,r) in stepthrough(m, policy, "s,b,a,o,r", max_steps=10)
    println("s: $s, b: $([pdf(b,s) for s in states(m)]), a: $a, o: $o")
    global rsum += r
end
println("Undiscounted reward was $rsum.")